In [ ]:
# default_exp qlearning.dqn_n_step

In [ ]:
#export
import torch.nn.utils as nn_utils
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.basics import *
from dataclasses import field,asdict
from typing import List,Any,Dict,Callable
from collections import deque
import gym
import torch.multiprocessing as mp
from torch.optim import *

from fastrl.data import *
from fastrl.async_data import *
from fastrl.basic_agents import *
from fastrl.learner import *
from fastrl.metrics import *
from fastrl.ptan_extension import *
from fastrl.qlearning.dqn import *

if IN_NOTEBOOK:
    from IPython import display
    import PIL.Image

/opt/conda/envs/fastrl/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


# DQN N-Steps

In [ ]:
# export
class DQNTrainer(Callback):
    def after_pred(self):
        s,a,r,sp,d,er,steps=(self.learn.xb+self.learn.yb)
        exps=[ExperienceFirstLast(*o) for o in zip(*(self.learn.xb+self.learn.yb))]
        batch_targets=[calc_target(self.learn.model, exp.reward, exp.last_state,exp.done,self.learn.discount**self.learn.n_steps)
                         for exp in exps]
        
        s_v = s.float()
        q_v = self.learn.model(s_v)
        t_q=q_v.data.numpy().copy()
        t_q[range(len(exps)), a] = batch_targets
        target_q_v = torch.tensor(t_q)
        self.learn._yb=self.learn.yb
        self.learn.yb=(target_q_v,)
        self.learn.pred=q_v
#         print(*self.learn.yb,self.learn.pred)
#         print(self.learn.pred,self.learn.yb)
#         print(self.learn._yb,self.learn.yb[0])
    
    def after_loss(self):self.learn.yb=self.learn._yb

In [ ]:
# export
class NStepDQNLearner(AgentLearner):
    def __init__(self,dls,discount=0.99,n_steps=3,**kwargs):
        store_attr()
        self.target_q_v=[]
        super().__init__(dls,loss_func=nn.MSELoss(),**kwargs)

In [ ]:
env='CartPole-v1'
model=LinearDQN((4,),2)
agent=DiscreteAgent(model=model.to(default_device()),device=default_device(),
                    a_selector=EpsilonGreedyActionSelector())

block=FirstLastExperienceBlock(agent=agent,seed=0,n_steps=3,dls_kwargs={'bs':32,'num_workers':0,'verbose':False,'indexed':True,'shuffle_train':False})
blk=IterableDataBlock(blocks=(block),
                      splitter=FuncSplitter(lambda x:False),
#                       batch_tfms=lambda x:(x['s'],x),
                     )
dls=blk.dataloaders([env]*1,n=32*1000,device=default_device())

learner=NStepDQNLearner(dls,agent=agent,n_steps=3,cbs=[EpsilonTracker,
                                        ExperienceReplay(sz=50000,bs=32,starting_els=32,max_steps=gym.make(env)._max_episode_steps),
                                        DQNTrainer],metrics=[AvgEpisodeRewardMetric(experience_cls=ExperienceFirstLast)])
learner.fit(15,lr=0.01,wd=0)

epoch,train_loss,train_avg_episode_r,valid_loss,valid_avg_episode_r,time
0,23.506865,31.350000,None,31.350000,00:12
1,38.347233,53.710000,None,53.710000,00:13
2,8.918954,92.310000,None,92.310000,00:12
3,3.378475,143.730000,None,143.730000,00:12
4,6.603476,194.220000,None,194.220000,00:13
5,4.584166,238.890000,None,238.890000,00:13
6,22.217869,266.190000,None,266.190000,00:12
7,8.952121,263.560000,None,263.560000,00:12
8,31.645321,241.610000,None,241.610000,00:13
9,2.677207,226.220000,None,226.220000,00:12


/opt/conda/envs/fastrl/lib/python3.7/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [ ]:
# hide
from nbdev.export import *
from nbdev.export2html import *
notebook2script()
notebook2html()

Converted 00_core.ipynb.
Converted 01_wrappers.ipynb.
Converted 03_basic_agents.ipynb.
Converted 04_learner.ipynb.
Converted 05a_ptan_extend.ipynb.
Converted 05b_data.ipynb.
Converted 05c_async_data.ipynb.
Converted 13_metrics.ipynb.
Converted 14a_actorcritic.sac.ipynb.
Converted 14b_actorcritic.diayn.ipynb.
Converted 15_actorcritic.a3c_data.ipynb.
Converted 16_actorcritic.a2c.ipynb.
Converted 17_actorcritc.v1.dads.ipynb.
Converted 18_policy_gradient.ppo.ipynb.
Converted 19_policy_gradient.trpo.ipynb.
Converted 20a_qlearning.dqn.ipynb.
Converted 20b_qlearning.dqn_n_step.ipynb.
Converted 20c_qlearning.dqn_target.ipynb.
Converted 20d_qlearning.dqn_double.ipynb.
Converted 20e_qlearning.dqn_noisy.ipynb.
Converted index.ipynb.
Converted notes.ipynb.


converting: /opt/project/fastrl/nbs/20b_qlearning.dqn_n_step.ipynb
converting: /opt/project/fastrl/nbs/14b_actorcritic.diayn.ipynb
An error occurred while executing the following cell:
------------------
from nbdev.showdoc import show_doc
from fastrl.actorcritic.diayn import *
------------------

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-1-906d7cb6bb2f> in <module>
      1 from nbdev.showdoc import show_doc
----> 2 from fastrl.actorcritic.diayn import *

AttributeError: module 'fastrl.actorcritic.diayn' has no attribute 'before_batch'
AttributeError: module 'fastrl.actorcritic.diayn' has no attribute 'before_batch'

Conversion failed on the following:
14b_actorcritic.diayn.ipynb
